# 1. Web Scraping: Scrape Table from a URL

#### Firstly import required libraries for scraping:

In [1]:
import csv
import requests
!pip install bs4 #install bs4 module
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M" #url of wikipedia page

#### Extract Table from Web page:

In [3]:
response = requests.get(url, timeout=10)
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.find_all('table')[0]
rows = table.select('tbody > tr')

In [4]:
header = [th.text.rstrip() for th in rows[0].find_all('th')]

#### Save the table as csv:

In [5]:
 with open('output.csv', 'w') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow(header)
        for row in rows[1:]:
            data = [th.text.rstrip() for th in row.find_all('td')]
            writer.writerow(data)


#### Clean table to get in required format using pandas:

In [6]:
df=pd.read_csv('output.csv')

In [7]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
df.shape

(288, 3)

In [9]:
df.set_index('Borough',inplace=True)

#####  a. Remove cells with a borough that is 'Not assigned':

In [10]:
df=df.drop('Not assigned')

In [11]:
df.shape

(211, 2)

In [12]:
df.reset_index(inplace=True)

In [13]:
df.set_index('Postcode',inplace=True)

In [14]:
df.head()

,Borough,Neighbourhood
Postcode,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,Harbourfront
M5A,Downtown Toronto,Regent Park
M6A,North York,Lawrence Heights


##### b. Similar rows are combined into one row with the neighborhoods separated with a comma:

In [15]:
df=df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()


In [16]:
df.head()


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


##### c. Cell that has a borough but a Not assigned neighborhood, then the neighborhood will changed to same as the borough

In [17]:
df[df['Neighbourhood']=='Not assigned']

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [18]:
df.Neighbourhood.replace(['Not assigned'],[df[df['Neighbourhood']=='Not assigned'].Borough],inplace=True)

##### d. Print the number of rows and columns of dataframe:

In [19]:
df.shape

(103, 3)

# 2. Add latitude and longitude to above dataframe

In [20]:
df1=pd.read_csv('Geospatial_Coordinates.csv')

In [21]:
df1.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [22]:
df1.columns=['Postcode', 'Latitude', 'Longitude']

In [23]:
df1.columns

Index(['Postcode', 'Latitude', 'Longitude'], dtype='object')

In [24]:
df2=pd.merge(df,df1,on="Postcode")

In [25]:
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


# Explore and cluster the neighborhoods in Toronto

##### a. import libraries used for exploring and clustring

In [26]:
import numpy as np 
import pandas as pd
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium 
print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


###### Get the Tornoto Geographical coordinates

In [27]:
address = 'toronto'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tornoto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tornoto are 43.653963, -79.387207.


#### Toronto map with all borough

In [28]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df2['Latitude'], df2['Longitude'], df2['Borough'], df2['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [29]:
tornoto_data = df2[df2.Borough.str.contains('Toronto')].reset_index(drop=True)
tornoto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


##### Toronto map with specific Borough

In [30]:
tornoto_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(tornoto_data['Latitude'], tornoto_data['Longitude'], tornoto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(tornoto_map) 
tornoto_map 

##### Foursquare Credentials and Version

In [31]:
CLIENT_ID = 'TR2J1EXMWNAAHLY5AUHLJZJQIHR2J2X3BL22NWXLJ2HLICQG' # your Foursquare ID
CLIENT_SECRET = 'MUSN2ELXO4YHLSYPVCNJCUJ2WA2Q2FX5RDL12EYUDVNROGUJ' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TR2J1EXMWNAAHLY5AUHLJZJQIHR2J2X3BL22NWXLJ2HLICQG
CLIENT_SECRET:MUSN2ELXO4YHLSYPVCNJCUJ2WA2Q2FX5RDL12EYUDVNROGUJ


In [32]:
LIMIT = 100
radius = 500

#### Let's create a function to get the neighborhoods in Tornoto

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
tornoto_venues = getNearbyVenues(names=tornoto_data['Neighbourhood'],
                                   latitudes=tornoto_data['Latitude'],
                                   longitudes=tornoto_data['Longitude']
                                  )



The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The 

##### Let's check the size of the resulting dataframe

In [35]:
print(tornoto_venues.shape)
tornoto_venues.head()

(1707, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


Let's check how many venues were returned for each neighborhood

In [36]:
tornoto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",16,16,16,16,16,16
"Cabbagetown, St. James Town",45,45,45,45,45,45
Central Bay Street,85,85,85,85,85,85
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


##### Let's find out how many unique categories can be curated from all the returned venues

In [37]:
print('There are {} uniques categories.'.format(len(tornoto_venues['Venue Category'].unique())))

There are 236 uniques categories.


## 3. Analyze Each Neighborhood

In [38]:
# one hot encoding
tornoto_onehot = pd.get_dummies(tornoto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
tornoto_onehot['Neighbourhood'] = tornoto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [tornoto_onehot.columns[-1]] + list(tornoto_onehot.columns[:-1])
tornoto_onehot = tornoto_onehot[fixed_columns]

tornoto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [45]:
tornoto_grouped = tornoto_onehot.groupby('Neighbourhood').mean().reset_index()
tornoto_grouped

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.030000,0.000000,...,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
1,Berczy Park,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.018182,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062500
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0625,0.0625,0.0625,0.125,0.1875,0.125,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,"Cabbagetown, St. James Town",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Bay Street,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.011765,0.000000,...,0.000000,0.000000,0.00,0.011765,0.000000,0.000000,0.011765,0.000000,0.000000,0.011765
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.010000,0.000000,0.00,0.060000,0.000000,0.040000,0.010000,0.000000,0.000000,0.000000
8,Christie,0.000000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000000,0.000000,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Church and Wellesley,0.010989,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.010989,0.000000,...,0.000000,0.000000,0.00,0.000000,0.010989,0.010989,0.000000,0.000000,0.010989,0.010989


##### Let's confirm the new size

In [46]:
tornoto_onehot.shape

(1707, 237)

##### Let's print each neighborhood along with the top 5 most common venues

In [47]:
num_top_venues = 5

for hood in tornoto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = tornoto_grouped[tornoto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
            venue  freq
0     Coffee Shop  0.08
1            Café  0.05
2      Steakhouse  0.04
3             Bar  0.04
4  Cosmetics Shop  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.07
1        Cocktail Bar  0.05
2  Italian Restaurant  0.04
3                Café  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.05
4   Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2         Auto Workshop  0.06
3         Garden Center  0.06
4  Fast Food Restaurant  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0     Airport Service  0.19
1      Airport Lounge

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [48]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [49]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = tornoto_grouped['Neighbourhood']

for ind in np.arange(tornoto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tornoto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Steakhouse,Bar,Cosmetics Shop,American Restaurant,Thai Restaurant,Hotel,Restaurant,Burger Joint
1,Berczy Park,Coffee Shop,Cocktail Bar,Beer Bar,Bakery,Steakhouse,Cheese Shop,Seafood Restaurant,Café,Italian Restaurant,Farmers Market
2,"Brockton, Exhibition Place, Parkdale Village",Coffee Shop,Breakfast Spot,Café,Bakery,Stadium,Burrito Place,Restaurant,Caribbean Restaurant,Climbing Gym,Yoga Studio
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Garden Center,Fast Food Restaurant,Farmers Market,Comic Shop,Recording Studio,Restaurant,Burrito Place
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Plane,Sculpture Garden,Coffee Shop,Harbor / Marina,Bar,Boat or Ferry,Airport Gate


## 4. Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

In [50]:
# set number of clusters
kclusters = 3

tornoto_grouped_clustering = tornoto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(tornoto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

tornoto_merged = tornoto_data




In [52]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
tornoto_merged = tornoto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

tornoto_merged.head()# check the last columns!


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Pub,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,2,Park,Pet Store,Burrito Place,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Fish & Chips Shop,Movie Theater,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Stationery Store,Clothing Store,Diner
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Construction & Landscaping,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store


In [53]:
tornoto_merged.shape

(38, 16)

Finally, let's visualize the resulting clusters

In [54]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(tornoto_merged['Latitude'], tornoto_merged['Longitude'], tornoto_merged['Neighbourhood'], tornoto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.

#### Cluster 1

In [55]:
tornoto_merged.loc[tornoto_merged['Cluster Labels'] == 0, tornoto_merged.columns[[1] + list(range(5, tornoto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,0,Construction & Landscaping,Park,Swim School,Bus Line,Yoga Studio,Diner,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
10,Downtown Toronto,0,Park,Trail,Playground,Building,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio


#### Cluster 2

In [56]:
tornoto_merged.loc[tornoto_merged['Cluster Labels'] == 1, tornoto_merged.columns[[1] + list(range(5, tornoto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,1,Garden,Yoga Studio,Dim Sum Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant


#### Cluster 3

In [57]:
tornoto_merged.loc[tornoto_merged['Cluster Labels'] == 2, tornoto_merged.columns[[1] + list(range(5, tornoto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,2,Pub,Neighborhood,Health Food Store,Other Great Outdoors,Trail,Yoga Studio,Dim Sum Restaurant,Diner,Discount Store,Dog Run
1,East Toronto,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Bookstore,Furniture / Home Store,Ice Cream Shop,Pizza Place,Brewery,Bubble Tea Shop,Restaurant
2,East Toronto,2,Park,Pet Store,Burrito Place,Sushi Restaurant,Brewery,Ice Cream Shop,Pub,Fish & Chips Shop,Movie Theater,Steakhouse
3,East Toronto,2,Café,Coffee Shop,American Restaurant,Italian Restaurant,Bakery,Yoga Studio,Chinese Restaurant,Stationery Store,Clothing Store,Diner
5,Central Toronto,2,Pizza Place,Breakfast Spot,Hotel,Food & Drink Shop,Dog Run,Park,Clothing Store,Sandwich Place,Gym,Antique Shop
6,Central Toronto,2,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Chinese Restaurant,Dessert Shop,Café,Rental Car Location,Restaurant,Diner
7,Central Toronto,2,Sandwich Place,Dessert Shop,Pizza Place,Gym,Coffee Shop,Café,Sushi Restaurant,Italian Restaurant,Thai Restaurant,Flower Shop
8,Central Toronto,2,Gym,Playground,Restaurant,Yoga Studio,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
9,Central Toronto,2,Coffee Shop,Pub,Bagel Shop,Light Rail Station,Supermarket,Vietnamese Restaurant,Sushi Restaurant,Pizza Place,Restaurant,American Restaurant
11,Downtown Toronto,2,Coffee Shop,Pub,Pizza Place,Italian Restaurant,Park,Restaurant,Café,Bakery,Jewelry Store,Japanese Restaurant
